In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import numpy as np
import pandas as pd
import sys

from sklearn.datasets import load_diabetes
sys.path.append("..")
from interpretDistill.fourierDistill import FTDistill, FTDistillCV
from interpretDistill.binaryTransformer import BinaryTransformer

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

from interpretDistill.FTutils import *

In [2]:
X, y = load_diabetes(as_frame = True, return_X_y = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.5, random_state=42)

In [3]:
X.columns

Index(['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6'], dtype='object')

In [4]:
bt_b = BinaryTransformer(depth=2, bit = True)
X_train_b = bt_b.fit_and_transform(X_train, y_train)
X_val_b = bt_b.transform(X_val)
# X_val = bt.fit_and_transform(X_val, y_val)
# X_train = bt.transform(X_train)
X_test_b = bt_b.transform(X_test)

[2, 3, 5, 6]
[2, 3, 5, 6]
[2, 3, 5, 6]
[2, 3, 5, 6]
[2, 3, 5, 6]
[2, 3, 5, 6]
[2, 3, 5, 6]
[2, 3, 5, 6]
[2, 3, 5, 6]
[2, 3, 5, 6]
[2, 3, 5, 6]
[2, 3, 5, 6]
[2, 3, 5, 6]
[2, 3, 5, 6]
[3, 5, 6]
[2, 3, 5, 6]
[2, 3, 5, 6]
[2, 3, 5, 6]
[2, 3, 5, 6]
[2, 3, 5, 6]
[2, 3, 5, 6]
[2, 3, 5, 6]
[2, 3, 5, 6]
[3, 5, 6]
[2, 3, 5, 6]
[2, 3, 5, 6]
[2, 3, 5, 6]


In [5]:
X_train_b.shape, X_val_b.shape, X_test_b.shape

((176, 37), (177, 37), (89, 37))

In [19]:
bt = BinaryTransformer(depth=3)
X_train_n = bt.fit_and_transform(X_train, y_train)
X_val_n = bt.transform(X_val)
# X_val = bt.fit_and_transform(X_val, y_val)
# X_train = bt.transform(X_train)
X_test_n = bt.transform(X_test)


[3, 4, 6, 7, 10, 11, 13, 14]
[3, 4, 6, 7, 10, 11, 13, 14]
[3, 4, 6, 7, 10, 11, 13, 14]
[3, 4, 6, 7, 10, 11, 13, 14]
[3, 4, 6, 7, 10, 11, 13, 14]
[2, 4, 5, 8, 9, 11, 12]
[2, 4, 5, 8, 9, 11, 12]
[3, 4, 6, 7, 10, 11, 13, 14]
[3, 4, 6, 7, 10, 11, 13, 14]
[3, 4, 6, 7, 10, 11, 13, 14]
[3, 4, 6, 7, 10, 11, 13, 14]
[3, 4, 7, 10, 11, 13, 14]
[4, 7, 10, 11, 13, 14]
[3, 6, 7, 10, 11, 13, 14]
[4, 5, 8, 9, 11, 12]
[2, 4, 5, 8, 9, 11, 12]
[3, 4, 6, 7, 10, 11, 13, 14]
[3, 4, 6, 7, 10, 11, 13, 14]
[3, 4, 6, 7, 10, 11, 13, 14]
[3, 4, 6, 7, 10, 11, 13, 14]
[3, 4, 7, 10, 11, 13]
[3, 4, 6, 7, 11, 13, 14]
[3, 6, 7, 10, 11, 13]
[4, 5, 8, 9, 11, 12]
[2, 4, 5, 8, 9, 12]
[3, 4, 6, 7, 10, 11, 13, 14]
[3, 4, 7, 10, 11, 14]


In [20]:
X_train_n.shape, X_val_n.shape, X_test_n.shape

((176, 71), (177, 71), (89, 71))

In [35]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
lb.fit(np.arange(15))

LabelBinarizer()

In [29]:
hi = {1:1, 3:5}

In [34]:
max([2])

2

In [36]:
lb.transform([1, 13])

array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]])

In [9]:
rf = RandomForestRegressor(max_depth=4, random_state=42)
rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=4, random_state=42)

In [10]:
r2_score(y_train, rf.predict(X_train)), r2_score(y_val, rf.predict(X_val)), r2_score(y_test, rf.predict(X_test))

(0.7343143256407813, 0.45577105002110385, 0.4606394652399125)

In [11]:
np.mean((y_train- rf.predict(X_train))**2), np.mean((y_val- rf.predict(X_val))**2), np.mean((y_test- rf.predict(X_test))**2)

(1588.4285648520963, 3357.032562787853, 2857.6139603200086)

In [12]:
#ftd = FTDistillCV(selection = 'L1L2', lam1_range=np.arange(0.01, 1.1, 0.33), lam2_range=np.arange(0.01, 1.1, 0.33), size_interactions = 3)
ftd = FTDistill(selection = 'L1L2', lam1=5, lam2=5, size_interactions = 6)
ftd.fit(X_val, rf.predict(X_val), bt.no_interaction)

In [13]:
np.mean((y_train- ftd.predict(X_train))**2), np.mean((y_val- ftd.predict(X_val))**2), np.mean((y_test- ftd.predict(X_test))**2)

(22313.94448401466, 23341.589470391813, 20053.46243109127)

In [14]:
r2_score(y_train, ftd.predict(X_train)), r2_score(y_val, ftd.predict(X_val)), r2_score(y_test, ftd.predict(X_test))

(-2.7323021752644117, -2.7840469196284547, -2.784992084589863)

In [15]:
#ftd = FTDistillCV(selection = 'L1L2', lam1_range=np.arange(0.01, 1.1, 0.33), lam2_range=np.arange(0.01, 1.1, 0.33), size_interactions = 3)
ftd1 = FTDistill(selection = 'L1L2', lam1=10, lam2=10, size_interactions = 6)
ftd1.fit(X_train, rf.predict(X_train))

In [16]:
np.sum(ftd1.regression_model.coef_ != 0)

1

In [17]:
np.mean((y_train- ftd1.predict(X_train))**2), np.mean((y_val- ftd1.predict(X_val))**2), np.mean((y_test- ftd1.predict(X_test))**2)

(25363.901107703594, 26465.47956493484, 22958.634422380477)

In [18]:
r2_score(y_train, ftd1.predict(X_train)), r2_score(y_val, ftd1.predict(X_val)), r2_score(y_test, ftd1.predict(X_test))

(-3.242447736897911, -3.2904797272360042, -3.333328963031028)